In [12]:
# basic libraries
import itertools
import re
import string

# data processing 
import collections
import numpy as np
import pandas as pd

# natural language pre-processing 
from textblob import TextBlob
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer, PorterStemmer

# ML pipelines
from transformers import pipeline

# sentiment analysis
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# data visualization
import seaborn as sb
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/macarthur/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/macarthur/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# read csv
df = pd.read_csv('../csv/data_clean.csv')
df_sent = df.copy()

def clean_links(df):
#replace URL of a text
    df_sent['text'] = df_sent['text'].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ')

clean_links(df_sent)
df_sent['text'].head()
df_sent['text'] = df_sent['text'].astype(str)

In [11]:
df_sent['time']

0        8
1        8
2       15
3       15
4       15
        ..
4935    16
4936    16
4937    14
4938    15
4939    21
Name: time, Length: 4940, dtype: int64

In [10]:
df_sent['day_name']

0          Sunday
1          Sunday
2          Sunday
3          Sunday
4          Sunday
          ...    
4935      Tuesday
4936      Tuesday
4937       Friday
4938       Friday
4939    Wednesday
Name: day_name, Length: 4940, dtype: object

### example cells for are testing hugging face model- remove when connected to real corpora

In [15]:
classifier = pipeline("text-classification",model='bhadresh-savani/albert-base-v2-emotion', return_all_scores=True)
prediction = classifier("I hate cheese", )
print(prediction)

[[{'label': 'sadness', 'score': 0.012733055278658867}, {'label': 'joy', 'score': 0.0031371070072054863}, {'label': 'love', 'score': 0.0029459993820637465}, {'label': 'anger', 'score': 0.9677979946136475}, {'label': 'fear', 'score': 0.012017112225294113}, {'label': 'surprise', 'score': 0.0013687432510778308}]]


In [105]:
albert_column_names = ["sadness", "joy", "love", "anger", "fear", "surprise"]
df_albert = pd.DataFrame(columns = albert_column_names)
score_sadness = []
score_joy = []
score_love = []
score_anger = []
score_fear = []
score_surprise = []
for i in df_sent['text']:
    pred = classifier(i)
    pred = pred[0]
    score_sadness.append(pred[0]['score'])
    score_joy.append(pred[1]['score'])
    score_love.append(pred[2]['score'])
    score_anger.append(pred[3]['score'])
    score_fear.append(pred[4]['score'])
    score_surprise.append(pred[5]['score'])
    # pred_l.append(pred[0]['socore'])

RuntimeError: The size of tensor a (1030) must match the size of tensor b (512) at non-singleton dimension 1

In [106]:
df_albert["sadness"] = score_sadness
df_albert["joy"] = score_joy
df_albert["love"] = score_love
df_albert["anger"] = score_anger
df_albert["fear"] = score_fear
df_albert["surprise"] = score_surprise

In [121]:
len(df_albert)

3868

In [125]:
# classifier encounters problem on this cell:
df_sent['text'][3868]

'Hellloooo….\n\nI try to scrape a webpage and it seems they have some protection measures in place.\nI can extract some info but after approx. 38 iterations over 38 different webpages I only get gibberish back in my soup (example below).\nIf I type in the request manually 5 seconds later, I get the soup as it should be…\nAny ideas?\nNext step I am thinking about is to rotate IPs. But maybe it is not that complicated and you guys had the same problem?!\n\nThanks\nJerrit\n\n`&lt;html&gt;&lt;head&gt;&lt;meta charset="utf-8"/&gt;&lt;script&gt;f1xx.v1xx=v1xx;f1xx[374148]=window;f1xx[647467]=e8NN(f1xx[374148]);f1xx[125983]=n3EE(f1xx[374148]);f1xx[210876]=(function(){var P6=2;for(;P6 !== 1;){switch(P6){case 2:return {w3:(function(v3){var v6=2;for(;v6 !== 10;){switch(v6){case 2:var O3=function(W3){var u6=2;for(;u6 !== 13;){switch(u6){case 2:var o3=[];u6=1;break;case 14:return E3;break;case 8:U3=o3.H8NN(function(){var Z6=2;for(;Z6 !== 1;){switch(Z6){case 2:return 0.5 - B8NN.P8NN();break;}}}).D8

In [109]:
# saving current progress for now
df_albert.to_csv('sentiment_temp_8-8-2021.zip')

# update a cell content